In [52]:
import os, fnmatch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib
import cv2
import glob


SAMPLES_PER_TAG = 200
yaml = []

def create_yaml_entry(tag, x_width, xmin, y_height, ymin, filename):
    anno = '''- annotations:
  - {class: %s, x_width: %d, xmin: %d, y_height: %d, ymin: %d}
  class: image
  filename: %s
''' % (tag, x_width, xmin, y_height, ymin, filename)
    return anno

scenery = './scenery/*'
annotated_images = './annotated_images/'

tags = {'red', 'green', 'yellow', 'red-green'}
directory = "./positives/*/{}/*".format("red")

S = []
scenery_files = glob.glob(scenery)
print("found %d files in scenery directory %s" % (len(scenery_files), scenery))

for s in scenery_files:
    img = plt.imread(s)
    img = cv2.resize(img, (600,800), interpolation=cv2.INTER_CUBIC)
    S.append(img)
    
S = np.array(S)

# Read X Vector

for tag in tags:
    directory = "./positives/*/{}/*".format(tag)

    rois_images = glob.glob(directory)
    
    R = []
    
    for r in rois_images:
        img = plt.imread(r)
        R.append(img)
    
    R = np.array(R)
    
    for i in range(SAMPLES_PER_TAG):
        r_rand = np.random.randint(0, len(R))
        s_rand = np.random.randint(0, len(S))

        roi = R[r_rand]
        scenery = S[s_rand]
    
        scale_down = np.random.randint(25, 100) / 100.
        roi = cv2.resize(roi, (0,0), fx=scale_down, fy=scale_down)
    
        annotated_image = np.copy(scenery)
        
        x_width = roi.shape[1]
        xmin = np.random.randint(10, scenery.shape[1] - x_width - 10)
        y_height = roi.shape[0]
        ymin = np.random.randint(10, scenery.shape[0] - y_height - 10)
        
        filename = "%s_%d.jpg" % (tag, i)
        
        annotated_image[ymin:ymin+y_height,xmin:xmin+x_width ] = roi
        
        plt.imsave(arr = annotated_image, fname = annotated_images + filename)
        
        yaml.append(create_yaml_entry(tag, x_width, xmin, y_height, ymin, filename))

with open("./annotation.yaml", "w") as y:
    for l in yaml:
        y.write(l)



found 11 files in scenery directory ./scenery/*
